TODO: write markdown

In [2]:
import csv, itertools, os, json, traceback
import dotenv
import numpy as np 
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [3]:
# Dotenv load stuff
# TODO: write dotenv instructions
dotenv.load_dotenv()
PROJECT_DIR = os.getenv('PROJECT_DIR')

# Go to proper dir
os.chdir(PROJECT_DIR)

In [4]:
def get_coords_info_from_one_line(json_entry):
    """
    TODO: write docstring
    """
    # img_id = json_entry['Image_ID']
    # latitude = json_entry['Latitude']
    # longitude = json_entry['Longitude']
    return (
        json_entry['Image_ID'], 
        json_entry['Latitude'],
        json_entry['Longitude']
        )
    # return (img_id, latitude, longitude)

In [20]:
EXIF_DATA_FILE = f'{PROJECT_DIR}/exif_data/exif_data_11_29_first_1000.json'
def get_date_data(exif_data_loc):

    with open(EXIF_DATA_FILE, 'r') as in_file:
        date_data = []
        rows = json.load(in_file)
        for row in rows:
            id = row['Image_ID']
            try:
                dto =  row['EXIF:DateTimeOriginal']
            except KeyError as e:
                dto = None
            try:
                mod_date = row['EXIF:ModifyDate']
            except KeyError as e:
                mod_date = None
            date_data.append((id, dto, mod_date))
        return date_data

date_data = get_date_data(EXIF_DATA_FILE)
date_data
   

[('27653319031', '2016:05:24 08:43:59', '2016:06:17 08:30:16'),
 ('22697429926', '2015:10:28 06:58:48', '2015:11:03 10:05:15'),
 ('6650348617', None, None),
 ('12337695235', '2013:07:13 01:12:58', '2014:02:06 18:52:24'),
 ('8432503596', '2013:01:26 07:59:13', '2013:01:31 21:08:24'),
 ('15629040035', '2014:10:25 21:10:16', '2014:10:26 17:00:10'),
 ('20610126982', '2013:08:24 05:50:51', '2015:08:16 21:33:14'),
 ('29359326241', '2016:09:03 03:59:47', '2016:09:03 22:18:26'),
 ('14095127199', '2014:05:25 04:59:00', '2014:05:26 21:31:57'),
 ('6219236142', None, None),
 ('8310392852', '2012:04:19 05:58:36', '2012:12:26 23:21:22'),
 ('23313968663', '2015:10:28 06:54:01', '2015:12:24 21:11:52'),
 ('9534473273', '2013:08:17 23:39:55', '2013:08:18 20:53:25'),
 ('14086415848', '2014:05:25 04:58:45', '2014:05:26 21:59:04'),
 ('5669373138', None, None),
 ('27534225480', '2016:06:06 01:57:13', '2016:06:21 20:29:04'),
 ('10486187245', '2013:10:19 02:46:01', '2013:10:24 21:46:54'),
 ('14191645299', '20

In [5]:
# Geolocator stuff
geolocator = Nominatim(user_agent="test_app")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1) # Delay needed or else will run into problems

In [6]:
# Here are all the unique keys from geolocator.reverse(s).raw:
#
# place_id
# licence
# osm_type
# osm_id
# lat
# lon
# display_name
# address
# boundingbox

In [7]:
csv_headers = ['amenity', 'building', 'tourism', 'place', 'house_number', 'emergency', 'leisure', 'quarter',
              'highway', 'historic', 'man_made', 'natural', 'road', 'farm', 'isolated_dwelling', 'neighbourhood', 
              'residential', 'suburb', 'city_district', 'town', 'locality', 'city', 'hamlet', 'village', 
              'municipality', 'county', 'state_district', 'district', 'province', 'state', 'region', 'postcode', 'country', 
              'country_code']
address_dict = {}
for key in csv_headers:
    address_dict[key] = None

In [8]:
illegal_country_codes = ['eg']
missing_from_illegal_country_codes = []
# TODO make this conditional on file existing
if True:
    mode = 'w'

EXIF_DATA_FILE = f'{PROJECT_DIR}/exif_data/exif_data_11_29_first_1000.json'
OUTPUT_FILE = f'{PROJECT_DIR}/output.csv'
ctr = 0
with open(EXIF_DATA_FILE, 'r') as in_file:
    with open(OUTPUT_FILE, mode) as out_file:
        writer = csv.writer(out_file)
        if mode == 'w':
                header_row = itertools.chain(['image_id'], csv_headers)
                writer.writerow(header_row)  

        rows = json.load(in_file)
        for row in rows:
            
            img_id = row['Image_ID']
            latitude = row['Latitude']
            longitude = row['Longitude']
              
            try:
                address_data = geolocator.reverse((latitude, longitude)).raw['address']
                this_address_dict = address_dict.copy()
                # Some images don't have country codes
                try:
                    country_code = address_data['country_code']
                except KeyError as key_exc:
                    country_code = None

                if country_code not in illegal_country_codes:
                    # There's probably a better way to do this, but I don't care
                    for key, value in address_data.items():
                        this_address_dict[key] = value
                    out_row = itertools.chain([img_id], this_address_dict.values())
                    print(f"Wrote row img {img_id} to #{ctr}")
                    writer.writerow(out_row)
                else:
                    print(f'Skipping write for image with id,country code {img_id},{country_code}')
                    missing_from_illegal_country_codes.append(img_id)
            except Exception as e:
                print(f'Write failed for image with id {img_id}')
                # traceback.print_exc()
                # exit()

            ctr+=1

Wrote row img 27653319031 to #0
Wrote row img 22697429926 to #1
Wrote row img 6650348617 to #2
Wrote row img 12337695235 to #3
Wrote row img 8432503596 to #4
Wrote row img 15629040035 to #5
Wrote row img 20610126982 to #6
Wrote row img 29359326241 to #7
Wrote row img 14095127199 to #8
Wrote row img 6219236142 to #9
Wrote row img 8310392852 to #10
Wrote row img 23313968663 to #11
Wrote row img 9534473273 to #12
Wrote row img 14086415848 to #13
Wrote row img 5669373138 to #14
Wrote row img 27534225480 to #15
Wrote row img 10486187245 to #16
Wrote row img 14191645299 to #17
Wrote row img 9587382078 to #18
Wrote row img 15035657071 to #19
Wrote row img 4335488443 to #20
Wrote row img 3888632330 to #21
Wrote row img 13844211075 to #22
Wrote row img 14124294348 to #23
Wrote row img 15720949851 to #24
Wrote row img 6132942800 to #25
Wrote row img 20067543468 to #26
Wrote row img 28069344081 to #27
Wrote row img 28168408525 to #28
Wrote row img 6357276861 to #29
Wrote row img 44645439685 to #3

In [9]:
# amenity
# building
# tourism
# place (before house_number)
# house_number
# emergency, leisure
# quarter
# man_made, highway, historic, natural (only indication is before road)
# highway (only indicatio is before road)
# natural (only indicator is before road)
# historic comes before road
# road
# farm (after road, before city_district)
# isolated_dwelling (between road and city)
# neighbourhood
# residential (before suburb)
# suburb
# city_district (before town, after suburb)
# town
# locality
# city
# hamlet
# village (sometimes before city, sometimes after)
# municipality
# county
# state_district
# province (before state or region)
# state
# region (after state_district, state - before postcode)
# postcode
# country
# # country_code

# csv_headers = ['amenity', 'building', 'tourism', 'place', 'house_number', 'emergency', 'leisure', 'quarter',
#               'highway', 'historic', 'man_made', 'natural', 'road', 'farm', 'isolated_dwelling', 'neighbourhood', 
#               'residential', 'suburb', 'city_district', 'town', 'locality', 'city', 'hamlet', 'village', 
#               'municipality', 'county', 'state_district', 'district', 'province', 'state', 'region', 'postcode', 'country', 
#               'country_code']

In [11]:
data = pd.read_csv(OUTPUT_FILE)
data

,image_id,amenity,building,tourism,place,house_number,emergency,leisure,quarter,highway,...,municipality,county,state_district,district,province,state,region,postcode,country,country_code
0,27653319031,NaN,NaN,NaN,NaN,NaN,Lifeguard Base,NaN,NaN,NaN,...,Lake View Township,Cook County,NaN,NaN,NaN,Illinois,NaN,60613,United States,us
1,22697429926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Southland District,NaN,NaN,NaN,Southland,NaN,9672,New Zealand / Aotearoa,nz
2,6650348617,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Auckland,NaN,NaN,New Zealand / Aotearoa,nz
3,12337695235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Ruapehu District,NaN,NaN,NaN,Manawatū-Whanganui,NaN,NaN,New Zealand / Aotearoa,nz
4,8432503596,NaN,NaN,NaN,NaN,360,NaN,NaN,NaN,NaN,...,NaN,Central Hawke's Bay District,NaN,NaN,NaN,Hawke's Bay,NaN,NaN,New Zealand / Aotearoa,nz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,7725570652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
993,8371944547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Highland,NaN,NaN,NaN,Scotland,NaN,NaN,United Kingdom,gb
994,31368823864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
995,24603827750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
us = data[data['country'] == 'United States']
us['city']

0           Chicago
30              NaN
207             NaN
228             NaN
364             NaN
380    Fort Collins
483             NaN
579             NaN
677             NaN
757       Kissimmee
982             NaN
Name: city, dtype: object